In [29]:
# when not using colab
import sys
sys.path.append('../src/QA')
import reload_recursive
%reload word_embed
from word_embed import WordEmbed

In [2]:
%reload data_prep
from data_prep import DataPrep

All these functions should be moved to their own module, im just doing it like this as a rough draft.

In [24]:
import torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [37]:
df = DataPrep.parse_tsv('../WikiQA-train.tsv')
question_doc_raw_train = DataPrep.convert_pd_to_json(df)

In [51]:
%reload feat_extract
from feat_extract import FeatExt

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/max/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/max/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/max/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
%reload embed_doc
from embed_doc import EmbedAndConcat

In [52]:
train_q_inputs, train_doc_inputs, train_doc_targets = EmbedAndConcat.get_unrolled_embeddings(question_doc_raw_train)

In [8]:
df_test = DataPrep.parse_tsv('../WikiQA-test.tsv')
question_doc_raw_test = DataPrep.convert_pd_to_json(df_test)
test_q_inputs, test_doc_inputs, test_doc_targets = get_unrolled_embeddings(question_doc_raw_test)

In [10]:
import numpy as np
q_max_len = max(get_max(train_q_inputs), get_max(test_q_inputs))
doc_max_len = max(get_max(train_doc_inputs), get_max(test_doc_inputs))

print(f"question max len: {q_max_len}")
print(f"Document max len: {doc_max_len}")
def cut(desired_size, lizt):
    for elt in lizt:
        if len(elt) > desired_size:
            del elt[desired_size:]

def pad(desired_size, lizt, target=False):
    # embedding size for all should be the same.
    if not target:
        emb_size = np.shape(lizt[0][0])[0]
    for i,_ in enumerate(lizt):
        elt_len = len(lizt[i])
        if elt_len < desired_size:
            if target: # append empty answer to target
                lizt[i].extend(['OOA'] * (desired_size - elt_len))
            else: # pad empty arrays
                lizt[i].extend([np.array([0] * emb_size) for x in range(elt_len, desired_size)])

def cut_pad_to(desired_size, lizt, target=False):
    cut(desired_size, lizt)
    pad(desired_size, lizt, target=target)


doc_len = 256
import copy
train_q_cop = copy.deepcopy(train_q_inputs)
test_q_cop = copy.deepcopy(test_q_inputs)
train_doc_cop = copy.deepcopy(train_doc_inputs)
test_doc_cop = copy.deepcopy(test_doc_inputs)
train_doc_targets_cop = copy.deepcopy(train_doc_targets)
test_doc_targets_cop = copy.deepcopy(test_doc_targets)
cut_pad_to(q_max_len, train_q_cop)
cut_pad_to(q_max_len, test_q_cop)
cut_pad_to(doc_len, train_doc_cop)
cut_pad_to(doc_len, test_doc_cop)
cut_pad_to(doc_len, train_doc_targets_cop, target=True)
cut_pad_to(doc_len, test_doc_targets_cop, target=True)
print(f"After padding train question min len {get_min(train_q_cop)}")
print(f"After padding test doc min len {get_min(test_doc_cop)}")

question max len: 27
Document max len: 1889
After padding train question min len 27
After padding test doc min len 256


In [11]:
# convert the target tags into integers
# only OOA and IOA for now
target2int = {"OOA": 0, "IOA": 1, "BOA": 1, "EOA": 1}
int2target = {0: "OOA", 1: "IOA", 2: "BOA", 3: "EOA"}
def convert_targets(raw_target_list):
    for i, targets in enumerate(raw_target_list):
        for j, _ in enumerate(targets):
            raw_target_list[i][j] = target2int[raw_target_list[i][j]]
            
convert_targets(train_doc_targets_cop)
convert_targets(test_doc_targets_cop)

In [12]:
# returns ratios of the number of class instances
def get_class_weights(raw_targets, classes):
    freqs = [0]*classes
    for i, targets in enumerate(raw_targets):
        for j, _ in enumerate(targets):
            freqs[raw_targets[i][j]] += 1
    biggest_class = max(freqs)
    return [biggest_class/x for x in freqs]

training_class_weights = get_class_weights(train_doc_targets_cop, 2)
print(training_class_weights)

[1.0, 17.082546461579525]


In [13]:
from torch.utils.data import TensorDataset
import torch
train_dataset = TensorDataset(torch.Tensor(train_q_cop), torch.Tensor(train_doc_cop), torch.LongTensor(train_doc_targets_cop))

from torch.utils.data import DataLoader
batch_size = 128
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) 

test_question_tensor = torch.Tensor(test_q_cop)
test_doc_tensor = torch.Tensor(test_doc_cop)
test_target_tensor = torch.LongTensor(test_doc_targets_cop)

/tmp/ipykernel_33889/2486513719.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525493953/work/torch/csrc/utils/tensor_new.cpp:230.)
  train_dataset = TensorDataset(torch.Tensor(train_q_cop), torch.Tensor(train_doc_cop), torch.LongTensor(train_doc_targets_cop))


In [14]:
q_embed_size = np.shape(train_q_cop[0][0])[0]
doc_embed_size = np.shape(train_doc_cop[0][0])[0]

In [15]:
del train_q_inputs
del test_q_inputs
del train_doc_inputs
del test_doc_inputs
del train_doc_targets
del test_doc_targets
del train_q_cop
del test_q_cop
del train_doc_cop
del test_doc_cop
del train_doc_targets_cop
del test_doc_targets_cop

In [16]:
import gc
gc.collect()

0

In [17]:
def train(input_question_tensor, input_document_tensor, target_tensor, question_model, document_model, question_optimizer, document_optimizer, criterion):
    loss = 0    

    question_optimizer.zero_grad()
    document_optimizer.zero_grad()
    # get output from question model
    question_output = question_model(input_question_tensor)

    # predictions from the document model with output from question for attention
    document_output = document_model(input_document_tensor, question_output)
    
    loss = criterion(document_output.view(-1, document_output.shape[-1]), target_tensor.view(-1))
    loss.backward()
    
    question_optimizer.step()
    document_optimizer.step()

    return loss.item()

In [18]:
import time
import math

# Helper functions for training
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [19]:
import random
def trainIters(question_model, document_model, n_iters, class_balance=None, print_every=5, plot_every=10, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    question_model_optimizer = optim.SGD(question_model.parameters(), lr=learning_rate)
    document_model_optimizer = optim.SGD(document_model.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss(weight=class_balance)
    
    for iter in range(1, n_iters):
        for question_input,doc_input,targets in train_loader:
            question_input = question_input.to(device)
            doc_input = doc_input.to(device)
            targets = targets.to(device)

            loss = train(question_input, doc_input, targets, question_model, document_model, question_model_optimizer, document_model_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

In [20]:
%reload model.QA_RNN
from model import QA_RNN
hidden_size = 100
doc_rnn_model = QA_RNN.DocumentModel(doc_embed_size, hidden_size, 2).to(device)
question_rnn_model = QA_RNN.QuestionModel(q_embed_size, hidden_size).to(device)

In [21]:
trainIters(question_rnn_model, doc_rnn_model, 30, torch.Tensor(training_class_weights))
#trainIters(question_rnn_model, doc_rnn_model, 30)

0m 9s (- 0m 46s) (5 16%) 11.4348
0m 18s (- 0m 37s) (10 33%) 10.9692
0m 28s (- 0m 28s) (15 50%) 10.6623
0m 37s (- 0m 18s) (20 66%) 10.4199
0m 46s (- 0m 9s) (25 83%) 10.2434


In [22]:
from sklearn.metrics import classification_report
def evaluate(input_question_tensor, input_document_tensor, target_tensor, question_model, document_model):
    with torch.no_grad():
        loss = 0
        
        question_output = question_model(input_question_tensor)
        
        document_output = document_model(input_document_tensor, question_output)
        predicts = torch.argmax(document_output.view(-1, document_output.shape[-1]), dim=-1)
        print(sum(predicts==0))
        print(classification_report(predicts, target_tensor.view(-1)))

In [23]:
evaluate(test_question_tensor, test_doc_tensor, test_target_tensor, question_rnn_model, doc_rnn_model)

tensor(56328)
              precision    recall  f1-score   support

           0       0.36      0.99      0.53     56328
           1       0.93      0.07      0.14    104952

    accuracy                           0.39    161280
   macro avg       0.65      0.53      0.34    161280
weighted avg       0.73      0.39      0.28    161280

